# ChatStep 聊天步骤组件

用于显示和管理聊天中的中间步骤组件，比如思维链中的步骤。该组件提供了对步骤状态的管理，包括挂起、运行中、成功和失败等状态，以及相应的标题和内容控制。

底层实现为`panel.chat.ChatStep`，参数基本一致，参考文档：https://panel.holoviz.org/reference/chat/ChatStep.html


In [1]:
##ignore
%load_ext vuepy
from panel_vuepy import vpanel


## 基本用法

基本的步骤组件初始化：


In [2]:
%%vuepy_run --plugins vpanel --show-code
<template>
  <PnChatStep/>
</template>
<script lang='py'>
</script>

{"vue": "<template>\n  <PnChatStep/>\n</template>\n<script lang='py'>\n</script>\n", "setup": ""}



通过 `stream` 方法对内容实现以下操作：
- 附加内容，支持`Markdown`、图像等任何内容
- 覆盖内容

标题也可以通过 `stream_title` 方法对标题实现类似操作。

In [3]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnColumn>
  <PnChatStep ref="step_ref" :width='200' />
  <PnButton name="Add Content" @click="add_content()" />
</PnColumn>
</template>
<script lang='py'>
import asyncio
import panel as pn
from vuepy import ref

step_ref = ref(None)

async def add_content():
    step = step_ref.value.unwrap()
    step.title = ('Thinking...')
    step.stream("Just thinking...")
    await asyncio.sleep(0.5)
    
    # Calling stream again will concatenate the text
    step.stream(" about `ChatStep`!")
    await asyncio.sleep(0.5)
    
    step.stream('clear', replace=True)
    await asyncio.sleep(0.5)
    
    step.title = ('Ok')
    step.append(pn.pane.Image("https://assets.holoviz.org/panel/samples/png_sample.png", width=50, height=50))
    
</script>

{"vue": "<template>\n<PnColumn>\n  <PnChatStep ref=\"step_ref\" :width='200' />\n  <PnButton name=\"Add Content\" @click=\"add_content()\" />\n</PnColumn>\n</template>\n<script lang='py'>\nimport asyncio\nimport panel as pn\nfrom vuepy import ref\n\nstep_ref = ref(None)\n\nasync def add_content():\n    step = step_ref.value.unwrap()\n    step.title = ('Thinking...')\n    step.stream(\"Just thinking...\")\n    await asyncio.sleep(0.5)\n    \n    # Calling stream again will concatenate the text\n    step.stream(\" about `ChatStep`!\")\n    await asyncio.sleep(0.5)\n    \n    step.stream('clear', replace=True)\n    await asyncio.sleep(0.5)\n    \n    step.title = ('Ok')\n    step.append(pn.pane.Image(\"https://assets.holoviz.org/panel/samples/png_sample.png\", width=50, height=50))\n    \n</script>\n", "setup": ""}


## Badges

默认头像是 `BooleanStatus` 组件，但可以通过提供 `default_badges` 进行更改。值可以是表情符号、图像、文本或 Panel 对象

In [4]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnChatStep 
    :default_badges='default_badges'
    status="success"
/>
</template>
<script lang='py'>

default_badges={
    "pending": "🤔",
    "running": "🏃",
    "success": "🎉",
    "failed": "😞",
}
</script>

{"vue": "<template>\n<PnChatStep \n    :default_badges='default_badges'\n    status=\"success\"\n/>\n</template>\n<script lang='py'>\n\ndefault_badges={\n    \"pending\": \"\ud83e\udd14\",\n    \"running\": \"\ud83c\udfc3\",\n    \"success\": \"\ud83c\udf89\",\n    \"failed\": \"\ud83d\ude1e\",\n}\n</script>\n", "setup": ""}



## 状态管理

为了显示该步骤正在处理，您可以将`status`设置为 `running` 并提供 `running_title`，使用 `success_title` 在成功时更新标题。

In [5]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnCol>
  <PnChatStep
    :width='300'
    status="running" 
    running_title="Processing this step..."
    success_title="Pretend job done!"
    ref="step_ref"
  />
  <PnButton name="Set status" @click="on_click()" />
</PnCol>
</template>
<script lang='py'>
from vuepy import ref
import time

step_ref = ref(None)

def on_click():
    step = step_ref.value.unwrap()
    step.stream("Pretending to do something.")
    time.sleep(1)
    step.status = "success"
</script>

{"vue": "<template>\n<PnCol>\n  <PnChatStep\n    :width='300'\n    status=\"running\" \n    running_title=\"Processing this step...\"\n    success_title=\"Pretend job done!\"\n    ref=\"step_ref\"\n  />\n  <PnButton name=\"Set status\" @click=\"on_click()\" />\n</PnCol>\n</template>\n<script lang='py'>\nfrom vuepy import ref\nimport time\n\nstep_ref = ref(None)\n\ndef on_click():\n    step = step_ref.value.unwrap()\n    step.stream(\"Pretending to do something.\")\n    time.sleep(1)\n    step.status = \"success\"\n</script>\n", "setup": ""}



## 错误处理

处理失败状态：


In [6]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnCol>
  <PnChatStep 
    running_title="Processing this step..." 
    success_title="Pretend job done!"
    ref="step_ref"
  />
  <PnButton name="Click" @click="on_click()" />
</PnCol>
</template>
<script lang='py'>
import time
from vuepy import ref

step_ref = ref(None)

def on_click():
    step = step_ref.value.unwrap()
    step.status = "running"
    try:
        step.stream("Breaking something")
        time.sleep(0.5)
        raise RuntimeError("Just demoing!")
    except RuntimeError as e:
        step.status = "failed"
        step.stream(f"Error: {str(e)}", replace=True)
</script>

{"vue": "<template>\n<PnCol>\n  <PnChatStep \n    running_title=\"Processing this step...\" \n    success_title=\"Pretend job done!\"\n    ref=\"step_ref\"\n  />\n  <PnButton name=\"Click\" @click=\"on_click()\" />\n</PnCol>\n</template>\n<script lang='py'>\nimport time\nfrom vuepy import ref\n\nstep_ref = ref(None)\n\ndef on_click():\n    step = step_ref.value.unwrap()\n    step.status = \"running\"\n    try:\n        step.stream(\"Breaking something\")\n        time.sleep(0.5)\n        raise RuntimeError(\"Just demoing!\")\n    except RuntimeError as e:\n        step.status = \"failed\"\n        step.stream(f\"Error: {str(e)}\", replace=True)\n</script>\n", "setup": ""}



## 标题流式显示

支持标题的流式更新：


In [7]:
%%vuepy_run --plugins vpanel --show-code
<template>
<PnColumn>
  <PnChatStep :width='200' ref="step_ref" />
  <PnButton name="Stream Title" @click="stream_title()" />
</PnColumn>
</template>
<script lang='py'>
from vuepy import ref
import time

step_ref = ref(None)

def stream_title():
    step = step_ref.value.unwrap()
    step.status = "running"
    for char in "It's streaming a title!":
        time.sleep(0.1)
        step.stream_title(char)
</script>

{"vue": "<template>\n<PnColumn>\n  <PnChatStep :width='200' ref=\"step_ref\" />\n  <PnButton name=\"Stream Title\" @click=\"stream_title()\" />\n</PnColumn>\n</template>\n<script lang='py'>\nfrom vuepy import ref\nimport time\n\nstep_ref = ref(None)\n\ndef stream_title():\n    step = step_ref.value.unwrap()\n    step.status = \"running\"\n    for char in \"It's streaming a title!\":\n        time.sleep(0.1)\n        step.stream_title(char)\n</script>\n", "setup": ""}



## API

### 核心属性
| 属性名                | 说明                                                                 | 类型                          | 默认值          |
|----------------------|--------------------------------------------------------------------|-----------------------------|----------------|
| collapsed_on_success | 成功时是否折叠卡片                                                   | ^[bool]                     | True           |
| context_exception    | 异常处理方式（"raise"-抛出/"summary"-摘要/"verbose"-完整追踪/"ignore"-忽略） | ^[str]                      | "raise"        |
| success_title        | 成功状态标题（未设置时使用最后对象的字符串）                            | ^[str]                      | None           |
| default_title        | 默认标题（其他标题未设置时使用）                                       | ^[str]                      | ""             |
| failed_title         | 失败状态标题                                                        | ^[str]                      | None           |
| margin               | 外边距（(垂直,水平)或(上,右,下,左)）                                  | ^[tuple]                    | (5,5,5,10)     |
| objects              | 聊天步骤内容列表（按列布局，通常应整体替换）                            | ^[list]                     | []             |
| pending_title        | 等待状态标题                                                        | ^[str]                      | None           |
| running_title        | 运行状态标题                                                        | ^[str]                      | None           |
| status               | 步骤状态（"pending"/"running"/"success"/"failed"）                   | ^[str]                      | "pending"      |

### 样式属性
| 属性名            | 说明                                                                 | 类型                          | 默认值        |
|------------------|--------------------------------------------------------------------|-----------------------------|--------------|
| collapsed        | 是否折叠内容                                                        | ^[bool]                     | False        |
| default_badges   | 状态徽章映射（键必须为'pending'/'running'/'success'/'failed'）        | ^[dict]                     | 系统默认徽章    |

### Events

| 事件名   | 说明           | 类型                                      |
| ------- | ------------- | ----------------------------------------- |
| status  | 状态改变时触发  | ^[Callable]`(status: str) -> None`       |

### Slots

| 插槽名   | 说明               |
| ------- | ----------------- |
| default | 自定义步骤内容      |
| title   | 自定义标题内容      |

### 方法

| 方法名         | 说明            | 参数                                    |
| ------------- | -------------- | --------------------------------------- |
| stream        | 流式添加内容     | value, replace=False                    |
| stream_title  | 流式更新标题     | value, replace=False, status="running"  |
| serialize     | 序列化内容       | -                                      |


In [8]:
##ignore
import panel as pn
pn.extension()

chat_step = pn.chat.ChatStep()
chat_step.stream("Some content...")
pn.Row(
    chat_step.controls(jslink=False), chat_step,
)

Row
    [0] Tabs
        [0] WidgetBox(margin=(5, 10), name='Controls')
            [0] StaticText(value='<b>Controls</b>')
            [1] ListInput(description='The list of c..., name='Objects', type=<class 'list'>, value=[Markdown(str, ...])
            [2] Select(description='Whether to add scrollbars..., name='Scroll', options=OrderedDict([('False', ...]), value=False)
            [3] IntInput(description='Max pixel distance f..., name='Auto scroll limit', start=0)
            [4] IntInput(description='Min pixel distance f..., name='Scroll button threshold', start=0)
            [5] IntInput(description='Current scroll p..., name='Scroll position')
            [6] Checkbox(name='View latest')
            [7] TextInput(description='A valid CSS c..., name='Active header b...)
            [8] ListInput(description='CSS classes t..., name='Button css classes', type=<class 'list'>, value=['card-button'])
            [9] Checkbox(name='Collapsible', value=True)
            [10] Checkbox(name='Collapsed')
            [11] Row(width=300)
                [0] LiteralInput(description='A Panel component t..., margin=(5, 0, 5, 10), name='Header', sizing_mode='stretch_width', value=Row)
                [1] Toggle(align='end', button_type='primary', height_policy='fit', margin=(0, 0, 5, 10), max_height=30, max_width=20, name='⋮')
            [12] TextInput(description='A valid CSS c..., name='Header background')
            [13] TextInput(description='A valid CSS c..., name='Header color')
            [14] ListInput(description='CSS classes t..., name='Header css classes', type=<class 'list'>, value=['card-header'])
            [15] Checkbox(name='Hide header')
            [16] ListInput(description='CSS classes t..., name='Title css classes', type=<class 'list'>, value=['card-title'])
            [17] TextInput(description='The title of the chat ste..., name='Title')
            [18] Checkbox(name='Collapsed on success', value=True)
            [19] Select(description='How to handle exceptions ..., name='Context exception', options=OrderedDict([('raise', ...]), value='raise')
            [20] DictInput(description="Mapping from status to de..., name='Default badges', type=<class 'dict'>, value={'pending': <function <lam...})
            [21] TextInput(description='The default t..., name='Default title')
            [22] TextInput(description='Title to display w..., name='Failed title')
            [23] TextInput(description='Title to display w..., name='Pending title')
            [24] TextInput(description='Title to display w..., name='Running title')
            [25] Select(name='Status', options=OrderedDict([('pending', ...]), value='pending')
            [26] TextInput(description='Title to display w..., name='Success title')
        [1] WidgetBox(margin=(5, 10), name='Layout')
            [0] StaticText(value='<b>Layout</b>')
            [1] TextInput(description='String identifier f..., name='Name', value='ChatStep00286')
            [2] LiteralInput(description='Whether the object should..., name='Align', value='start')
            [3] LiteralInput(description='Describes the proportiona..., name='Aspect ratio')
            [4] ListInput(description='CSS classes t..., name='Css classes', type=<class 'list'>, value=['card'])
            [5] Select(description='The design system t..., name='Design', options=OrderedDict([('None', ...]))
            [6] IntInput(description='The height of the compone..., name='Height', start=0)
            [7] IntInput(description='Minimal width o..., name='Min width', start=0)
            [8] IntInput(description='Minimal height o..., name='Min height', start=0)
            [9] IntInput(description='Maximum width o..., name='Max width', start=0)
            [10] IntInput(description='Maximum height o..., name='Max height', start=0)
            [11] LiteralInput(description='Allows to create addition..., name='Margin', value=(5, 5, 5, 10))
            [12] DictInput(description='Dictionary of C..., 